In [ ]:
!nvidia-smi

Fri Nov 24 07:01:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
## Install dependencies
!pip install datasets transformers sentencepiece accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00


In [ ]:
import os
import sys
import transformers
import datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AdamWeightDecay

## Load Dataset

In [ ]:
from datasets import get_dataset_split_names

get_dataset_split_names(path="ai4bharat/samanantar", config_name="as")

['train']

In [ ]:
from datasets import get_dataset_config_names

configs = get_dataset_config_names("ai4bharat/samanantar")
print(configs)

['as', 'bn', 'gu', 'hi', 'kn', 'ml', 'mr', 'or', 'pa', 'ta', 'te']


In [ ]:
from datasets import load_dataset
dataset = load_dataset("ai4bharat/samanantar", "as")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'src', 'tgt'],
        num_rows: 141227
    })
})

In [ ]:
dataset['train'][1]

{'idx': 1,
 'src': 'Nevertheless, he gave this assurance: He that has endured to the end is the one that will be saved.',
 'tgt': 'যিয়েই নহওঁক, যিসকলে এই কাৰ্য্য প্ৰাণপণে কৰা চেষ্টা কৰিব, তেওঁলোকক আশ্বাস দি যীচুৱে এইদৰে কৈছিল: “যি জনে শেষলৈকে সহি থাকে, সেই জনেই পৰিত্ৰাণ পাব । ”'}

In [ ]:
dataset.save_to_disk("data")

Saving the dataset (0/1 shards):   0%|          | 0/141227 [00:00<?, ? examples/s]

## PreProcessing the data

In [ ]:
checkpoint = "Helsinki-NLP/opus-tatoeba-en-ro"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/792k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/817k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenizer(["My name is Wolfgang and I live in Berlin", "hi thre"])

{'input_ids': [[1577, 528, 31, 55686, 15, 19, 791, 17, 7446, 0], [4430, 9319, 76, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
  print(tokenizer(["Numele meu este Wolfgang și locuiesc în Berlin"]))

{'input_ids': [[2585, 245, 50, 12188, 399, 2698, 40, 10360, 13, 7446, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
max_input_length = 128
max_target_length = 128



def preprocess_function(data):
  inputs = [ex for ex in data['src']]
  targets = [ex for ex in data['tgt']]

  model_inputs = tokenizer(inputs, max_length=max_input_length,padding="max_length", truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length,padding="max_length", truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

preprocess_function(dataset['train'][:2])


{'input_ids': [[625, 470, 110, 492, 2298, 2, 0, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000], [51909, 3, 89, 1371, 57, 17032, 56, 590, 28, 155, 43251, 9, 4, 755, 31, 4, 124, 28, 106, 45, 4178, 2, 0,

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/141227 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['idx', 'src', 'tgt', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 141227
    })
})

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=20).select(range(200))

## Download Model

In [ ]:
checkpoint = "Helsinki-NLP/opus-tatoeba-en-ro"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/155M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
model.save_pretrained("models/")

## Training

In [ ]:
!pip install accelerate

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored